## Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import copy
import torch
import loss
from torch import optim
from metrics import eval_metrics, get_epoch_acc
from dataloader import DataLoader
from cross_val import CrossVal
from torchvision import transforms
from eval import eval
from config import ModelParameters, DatasetConfig

# Import available models, you can also explore other PyTorch models
from cracknet import cracknet
from unet import UNet, UNetResnet
from segnet import SegNet, SegResNet

# https://blog.roboflow.com/how-to-use-segment-anything-model-sam/
from segment_anything import sam_model_registry

2024-11-17 11:10:43.587650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731841843.608383  158919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731841843.614813  158919 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 11:10:43.636370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# os.environ["TORCH_USE_CUDA_DSA"] = "1"

## Training functions

In [3]:
def train_oneepoch(model, class_count, criterion, eval_metric, device, my_optimizer, my_lr_scheduler, dataloader):
    model.train()
    batch_loss = 0
    batch_acc_numerator = 0
    batch_acc_denominator = 0
    epoch_lr = my_lr_scheduler.get_last_lr()[0]
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        my_optimizer.zero_grad()
        mask_pred = model(inputs)
        loss = criterion(mask_pred, labels)
        loss.backward()
        my_optimizer.step()
        
        # batch_loss += loss
        batch_loss += loss.item()
        
        batch_acc_numerator_tmp, batch_acc_denominator_tmp = eval_metrics(mask_pred, labels, class_count, eval_metric)
        batch_acc_numerator += batch_acc_numerator_tmp
        batch_acc_denominator += batch_acc_denominator_tmp
    my_lr_scheduler.step()
    epoch_loss = batch_loss / len(dataloader)
    epoch_acc = get_epoch_acc(batch_acc_numerator, batch_acc_denominator, eval_metric)

    return epoch_loss, epoch_acc, epoch_lr

In [4]:
def train_main(model, class_count, criterion, eval_metric, EPOCHS, DEVICE, my_optimizer, my_lr_scheduler=None, dataloaders=None, logging=False, model_name='model.pt'):
    model.to(DEVICE)
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    lr = []
    best_val_acc = 0

    for epoch in range(EPOCHS):
        epoch_train_loss, epoch_train_acc, epoch_lr = train_oneepoch(model, class_count, criterion, eval_metric, DEVICE, my_optimizer, my_lr_scheduler, dataloaders['train'])
        epoch_val_loss, epoch_val_acc = eval(model, class_count, criterion, eval_metric, DEVICE, dataloaders['val'])
        # print(epoch_val_loss)
        print(type(epoch_val_acc))
        print(epoch_val_acc)
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
        best_state_dict = copy.deepcopy(model.state_dict())

        if logging:
            # train_loss.append(epoch_train_loss.detach().cpu().numpy().tolist())
            train_loss.append(epoch_train_loss)
            train_acc.append(epoch_train_acc)
            # val_loss.append(epoch_val_loss.detach().cpu().numpy().tolist())
            val_loss.append(epoch_val_loss)
            val_acc.append(epoch_val_acc)
            lr.append(epoch_lr)
        torch.cuda.empty_cache() 
        
        print(f'Epoch {epoch}/{EPOCHS - 1}: TrainLoss: {epoch_train_loss:.4f}, TrainAcc: {epoch_train_acc:.4f}, ValLoss: {epoch_val_loss:.4f}, ValAcc: {epoch_val_acc:.4f}')

    print('Best Acc: {:4f}'.format(best_val_acc))

    # load best model weights
    model.load_state_dict(best_state_dict)
    torch.save(model, model_name + '.pt')
    
    # save training details
    pd.DataFrame({'Epochs':range(EPOCHS), 'Learning Rate': lr, 'Training Loss': train_loss, 
                    'Training Acc': train_acc, 'Validation Loss': val_loss, 
                    'Validation Acc': val_acc}).to_csv(model_name + '.csv', index = False)

    return model

## Training parameters

Inside the data directory, the structure should be following:
- train
    - images
        - IL 991.png
        - IL 992.png
    - labels
        - IL 991.npy
        - IL 992.npy
    - class_names.txt
- val
    - images
        - IL 993.png
    - labels
        - IL 993.npy
    - class_names.txt

class_names.txt is to specify the label class name for the training

Example content of class_names.txt:

\_background_ <br>
fault

In [5]:
# Name the data directory and model filename
DIR = 'data/' # Data directory
MODEL_FILENAME = 'cracknet.pt' # Model filename



In [6]:
num_fault = []
num_horizon = []
num_seismic = []
indeces = [v for v in range(1000, 1441)]
for name in os.listdir('../data/raw_fault'):
    if name == '.ipynb_checkpoints':
        continue

    num_fault.append(int(name.split('-')[1].split('.')[0]))

for name in os.listdir('../data/raw_horizon'):
    if name == '.ipynb_checkpoints':
        continue
        
    num_horizon.append(int(name.split('-')[1].split('.')[0]))

for name in os.listdir('../data/raw_seismic'):
    if name == '.ipynb_checkpoints':
        continue
        
    num_seismic.append(int(name.split('-')[1].split('.')[0]))

df = pd.DataFrame({
    'RAW_SEISMIC': [f"seismic-{x}.png" if x in num_seismic else None for x in indeces],
    'RAW_FAULT': [f"fault-{x}.npy" if x in num_fault else None for x in indeces],
    'RAW_HORIZON': [f"horizon-{x}.npy" if x in num_horizon else None for x in indeces]
})

In [7]:
cv = CrossVal(df, 3)
dataloaders = cv[0]
# dataset = {}
# dataset['train'] = LabelMe(data_folder=os.path.join(DIR,'train'), transform=data_transforms['train'],
#                                 img_size=(1024, 1024))
# dataset['val'] = LabelMe(data_folder=os.path.join(DIR,'val'), transform=data_transforms['val'],
#                                 img_size=(1024, 1024))
# dataloaders = {x: torch.utils.data.DataLoader(dataset[x], batch_size = BATCH_SIZE,
#                                             shuffle = True, num_workers = 8, 
#                                             drop_last = False)
#                                             for x in ['train', 'val']}
class_count = len(dataloaders['train'].dataset.label)

In [8]:
# Choose a model for training, you can refer to the models that have been imported above
# model = cracknet(pretrained = ModelParameters.PRETRAINED, num_classes = class_count)
model = UNet(num_classes = class_count)

# MODEL_TYPE = "vit_h"
# CHECKPOINT_PATH = "meta_pretrained/sam_vit_h_4b8939.pth"
# model = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
# # sam.to(device=DEVICE)

my_optimizer = optim.Adam(model.parameters(), lr = ModelParameters.LEARNING_RATE) # Check https://pytorch.org/docs/stable/optim.html for other optimizers
my_lr_scheduler = optim.lr_scheduler.StepLR(my_optimizer, step_size=25, gamma=0.1) # Check https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate for other schedulers
# my_lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(my_optimizer,
#                                                       factor=0.1, 
#                                                       patience=10,
#                                                       threshold=0.000001
#                                                       ) # Check https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate for other schedulers

## Start model training

In [9]:
train_main(model, class_count, ModelParameters.CRITERION, ModelParameters.EVAL_METRIC,
           # 1,                  # Epochs
           ModelParameters.EPOCHS,
           DEVICE, my_optimizer, 
           my_lr_scheduler, dataloaders, 
           logging = ModelParameters.LOGGING, 
           model_name = MODEL_FILENAME
          )

########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torch.Size([2, 3, 512, 512])
########
########
torc

UNet(
  (start_conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (down1): encoder(
    (down_conv): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mod

In [10]:
cracknet_model = cracknet(pretrained = ModelParameters.PRETRAINED, num_classes = class_count)
cracknet_model_total_params = sum(p.numel() for p in cracknet_model.parameters())

unet_model = UNet(num_classes = class_count)
unet_model_total_params = sum(p.numel() for p in unet_model.parameters())

unet_resnet_model = UNetResnet(num_classes = class_count)
unet_resnet_model_total_params = sum(p.numel() for p in unet_resnet_model.parameters())

segnet_model = SegNet(num_classes = class_count)
segnet_model_total_params  = sum(p.numel() for p in segnet_model.parameters())

segresnet_model = SegResNet(num_classes = class_count)
segresnet_model_total_params  = sum(p.numel() for p in segresnet_model.parameters())

/home/participant4/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/participant4/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/participant4/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.D

In [11]:
cracknet_model_total_params

49320390

In [12]:
unet_model_total_params

26355234

In [13]:
unet_resnet_model_total_params

29999920

In [14]:
segnet_model_total_params

16310850

In [15]:
segresnet_model_total_params

53553346